In [1]:
import pandas as pd
train = pd.read_csv('../input/tps-apr-22-fe-v2/train_tps_apr_22_fev2.csv')
test = pd.read_csv('../input/tps-apr-22-fe-v2/test_tps_apr_22_fev2.csv')

In [2]:
groups = train['subject']
labels = train['state']
train.drop(['sequence', 'subject', 'state'], axis=1, inplace=True)
train.head()

# LightGBMError: Do not support special JSON characters in feature name.
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [3]:
params = {'n_estimators': 16500, 
          'learning_rate': 0.06946446628215118, 
          'reg_alpha': 0.010253222571604808, 
          'reg_lambda': 0.031961179402523146, 
          'colsample_bytree': 1.0, 
          'subsample': 0.6, 
          'max_depth': 100, 
          'num_leaves': 867, 
          'min_child_samples': 291, 
          'min_data_per_groups': 26}

In [4]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

kf = GroupKFold(n_splits=5)
scores = []

for fold_idx, (train_index, test_index) in enumerate(kf.split(train, labels, groups)):
    X_train_group = train.iloc[train_index] 
    X_valid_group = train.iloc[test_index]
    y_train_group = labels.iloc[train_index] 
    y_valid_group = labels.iloc[test_index]
    
    model = lgbm.LGBMClassifier(**params)
    model.fit(X_train_group, y_train_group, eval_set = [(X_valid_group, y_valid_group)], 
                early_stopping_rounds=300, verbose=False)
    fold_score = roc_auc_score(y_valid_group, model.predict_proba(X_valid_group)[:, 1])
    scores.append(fold_score)
    
    print(f'kfold: {fold_idx + 1} , Fold score: {fold_score}')
    
print(f'cv score: {sum(scores) / 5}')

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
kfold: 1 , Fold score: 0.9287844891871738
[LightGBM] [Warning] Unknown parameter: min_data_per_groups


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


kfold: 2 , Fold score: 0.9278974457859919
[LightGBM] [Warning] Unknown parameter: min_data_per_groups


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


kfold: 3 , Fold score: 0.9356581464658085
[LightGBM] [Warning] Unknown parameter: min_data_per_groups


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


kfold: 4 , Fold score: 0.9296936028570211
[LightGBM] [Warning] Unknown parameter: min_data_per_groups


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


kfold: 5 , Fold score: 0.9301350860903854
cv score: 0.930433754077276


In [5]:
submission = test[['sequence']]
predictions = pd.DataFrame(
    [model.predict_proba(test.drop(['sequence', 'subject'], axis=1))[:, 1]]).T

submission['state'] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,sequence,state
0,25968,0.984161
1,26059,0.603828
2,26176,0.904114
3,26249,0.995387
4,26258,0.977215
